In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)

import numpy as np

import sys
sys.path.insert(0, '/home/storage/hans/jax_reco_gupta_corrections/')

from lib.geo import __sin_theta_cherenkov, __tan_theta_cherenkov, get_xyz_from_zenith_azimuth

In [3]:
def impact_angle_cos_eta(dom_pos, track_pos, track_dir):
    """
    dom_pos: 1D jax array with 3 components [x, y, z]
    track_pos: 1D jax array with 3 components [x, y, z]
    track_dir: 1D jax array with 3 components [dir_x, dir_y, dir_z]
    """
    # vector from vertex to dom
    v_a = dom_pos - track_pos

    # distance muon travels from track vertex to point of closest approach.
    ds = jnp.dot(v_a, track_dir)
    print("distance the muon travels:", ds)

    # a vector parallel track with length ds
    ds_v = ds * track_dir

    # vector closest approach position to dom yields closest approach distance
    v_d = v_a - ds_v
    dc = jnp.linalg.norm(v_d)
    print("closest approach distance:", dc)

    # distance that the photon travels
    dt = dc / __sin_theta_cherenkov
    print("the photon travels:", dt)

    # distance emission point to closest approach point
    dx = dc / __tan_theta_cherenkov
    
    # distance vertex to emission point
    dy = ds - dx
    print("distance vertex to emission point:", dy)
    
    # z component of emission point
    z_emission = track_pos[2] + dy * track_dir[2] 
    print("z component of emission point:", z_emission)
    
    # cos eta
    return (dom_pos[2] - z_emission) / dt


In [4]:
track_pos = np.array([-10., 10., 200.])
track_zenith = np.deg2rad(30)
track_azimuth = np.deg2rad(2.)
dom_pos = np.array([-10, 10, -150])

track_dir = get_xyz_from_zenith_azimuth(jnp.array([track_zenith, track_azimuth]))
print(track_dir)

[-0.49969541 -0.01744975 -0.8660254 ]


In [5]:
cos_eta = impact_angle_cos_eta(dom_pos, track_pos, track_dir)
print(cos_eta, np.rad2deg(np.arccos(cos_eta)))

distance the muon travels: 303.10889132455355
closest approach distance: 174.99999999999997
the photon travels: 271.4912523387105
distance vertex to emission point: 95.54564771807725
z component of emission point: 117.25504185510641
-0.9843965120529222 169.86520877746315


In [6]:
from lib.geo import impact_angle_cos_eta

In [7]:
cos_eta = impact_angle_cos_eta(dom_pos, track_pos, track_dir)
print(cos_eta, np.rad2deg(np.arccos(cos_eta)))

distance the muon travels: 303.10889132455355
closest approach distance: 174.99999999999997
the photon travels: 271.4912523387105
distance vertex to emission point: 95.54564771807725
z component of emission point: 117.25504185510641
-0.9843965120529222 169.86520877746315
